In [1]:
import pandas as pd
import numpy as np

In [2]:
df= pd.read_csv('ex.csv ')
df.head(3)

,Song-Name,Singer/Artists,Genre,Album/Movie,User-Rating
0,Aankh Marey,"Kumar Sanu, Mika Singh, Neha Kakkar",BollywoodDance,Simmba,8.8/10
1,Coca Cola,"Neha Kakkar, Tony Kakkar",BollywoodDanceRomantic,Luka Chuppi,9.0/10
2,Apna Time Aayega,Ranveer Singh,BollywoodDance,Gully Boy,9.7/10


In [3]:
df.shape

(2420, 5)

In [4]:
df = df.sample(n=11).drop('Genre',axis=1).reset_index(drop=True)

In [5]:
#cleaning

In [6]:
df['Song-Name'][0]

'Maahi Ve'

In [7]:
df['Song-Name']= df['Song-Name'].str.lower().replace(r'[^\w\s]','').replace(r'\n',' ',regex=True)

In [8]:
df['Song-Name'][0]

'maahi ve'

In [9]:
import nltk
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()

def tokenization(txt):
    tokens= nltk.word_tokenize(txt)
    stemming = [ps.stem(w) for w in tokens]
    
    return " ".join(stemming)

In [10]:
df['Song-Name'] = df['Song-Name'].apply(lambda x: tokenization(x))

In [11]:
df['Song-Name']

0                            maahi ve
1                tum saath ho jab apn
2           woh ladki bahut yaad aati
3     band mein tha dum… vand mataram
4                    lamhon ke rasgul
5                       ishq da sutta
6               hori khele raghuveera
7                            ishq hua
8             tauba yeh matwali chaal
9                    jhalak dikhla ja
10                               heer
Name: Song-Name, dtype: object

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [25]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['Song-Name'])

In [26]:
matrix.shape

(11, 36)

In [27]:
similarity = cosine_similarity(matrix)

In [28]:
similarity[0]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [29]:
df['Song-Name'][0]

'maahi ve'

In [30]:
df[df['Song-Name']== 'maahi ve']

,Song-Name,Singer/Artists,Album/Movie,User-Rating
0,maahi ve,"Richa Sharma, Sukhwinder Singh",Kaante,9.2/10


In [31]:
sorted(list(enumerate(similarity[0])),reverse=False,key= lambda x:x[1])

[(1, 0.0),
 (2, 0.0),
 (3, 0.0),
 (4, 0.0),
 (5, 0.0),
 (6, 0.0),
 (7, 0.0),
 (8, 0.0),
 (9, 0.0),
 (10, 0.0),
 (0, 1.0000000000000002)]

In [32]:
def recommendation(song_df):
    idx = df[df['Song-Name'] == song_df].index[0]
    distance = sorted(list(enumerate(similarity[0])), reverse=False, key=lambda x: x[1])
    songs = []
    for i in distance[1:10]:
        songs.append(df.iloc[i[0]]['Song-Name'])
    return songs

In [33]:
recommendation('maahi ve')

['woh ladki bahut yaad aati',
 'band mein tha dum… vand mataram',
 'lamhon ke rasgul',
 'ishq da sutta',
 'hori khele raghuveera',
 'ishq hua',
 'tauba yeh matwali chaal',
 'jhalak dikhla ja',
 'heer']

In [34]:
import pickle

# Load pickled objects
with open('similarity.pkl', 'rb') as similarity_file:
    similarity = pickle.load(similarity_file)

with open('df.pkl', 'rb') as df_file:
    df = pickle.load(df_file)

# Now you can use the loaded objects (similarity and df) as needed.
